# LangGraph CLI Tool-Call Model Training Dataset Generator

This notebook generates a synthetic dataset of natural language queries and their corresponding structured JSON CLI commands for training a model to convert user requests into LangGraph CLI tool calls.

## Overview

We use **NVIDIA's NeMo Data Designer** (open-source via Docker) to:
1. Define a data schema with sampler columns for CLI parameters
2. Generate natural language queries using Nemotron
3. Create structured JSON output for each query
4. Export the dataset for model fine-tuning

### LangGraph CLI Commands Covered

| Command | Description |
|---------|-------------|
| `langgraph new` | Create a new LangGraph project from a template |
| `langgraph dev` | Run the development server with hot reload |
| `langgraph up` | Launch the server in a Docker container |
| `langgraph build` | Build the Docker image for deployment |
| `langgraph dockerfile` | Generate a Dockerfile for the project |


## Step 1: Launch NeMo Data Designer via Docker Compose

> ⚠️ **Before running this notebook**, follow the setup instructions in [README.md](./README.md) to start the NeMo Data Designer Docker service.

The cells below are provided for convenience if you haven't started the service yet.


In [ ]:
# Authenticate with NGC using your API key
!echo $NGC_CLI_API_KEY | docker login nvcr.io -u '$oauthtoken' --password-stdin


In [ ]:
# Download the NeMo Data Designer Docker Compose configuration
!ngc registry resource download-version "nvidia/nemo-microservices/nemo-data-designer-docker-compose:25.08"


In [ ]:
# Start the Data Designer services (runs on port 8000)
!cd nemo-data-designer-docker-compose_v25.08 && docker compose -f docker-compose.ea.yaml up -d


In [ ]:
# Wait for services to be ready (optional health check)
import time
import requests

print("Waiting for Data Designer service to be ready...")
max_retries = 30
for i in range(max_retries):
    try:
        response = requests.get("http://localhost:8000/health", timeout=5)
        if response.status_code == 200:
            print("✓ Data Designer service is ready!")
            break
    except requests.exceptions.RequestException:
        pass
    time.sleep(10)
    print(f"  Retry {i+1}/{max_retries}...")
else:
    print("⚠ Service may not be ready. Proceeding anyway...")


## Step 2: Install NeMo Data Designer SDK

Install the SDK and connect to the local Data Designer service.


In [ ]:
%pip install "nemo-microservices[data-designer]" -q


In [ ]:
from nemo_microservices.data_designer.essentials import (
    DataDesignerConfigBuilder,
    LLMTextColumnConfig,
    SamplerColumnConfig,
    SamplerType,
    CategorySamplerParams,
    BooleanSamplerParams,
    UniformSamplerParams,
    NeMoDataDesignerClient
)

# Connect to local Data Designer service (no API key needed for local)
client = NeMoDataDesignerClient(base_url="http://localhost:8000/v1/nemo/dd")
print("✓ Connected to NeMo Data Designer")


## Step 3: Define Schema for Synthetic Dataset

We define a schema with:
- **Sampler columns**: Randomly generated values for CLI parameters
- **LLM text columns**: Natural language and structured JSON generated by Nemotron

### Schema Overview

| Column | Type | Description |
|--------|------|-------------|
| `command` | Category | One of: new, dev, up, build, dockerfile |
| `template` | Category | Project template for 'new' command |
| `include_path` | Boolean | Whether user specifies a custom path |
| `port` | Uniform Int | Port number (3000-9000) |
| `no_browser` | Boolean | Skip opening browser |
| `watch` | Boolean | Watch for code changes |
| `image_tag` | Category | Docker image tag |
| `dockerfile_path` | Category | Custom Dockerfile path |
| `input` | LLM Text | Natural language user query |
| `output` | LLM Text | Structured JSON tool-call |


In [ ]:
# Initialize the configuration builder
config_builder = DataDesignerConfigBuilder()


In [ ]:
# Column 1: CLI command type (randomly choose one of the 5 LangGraph CLI commands)
config_builder.add_column(
    SamplerColumnConfig(
        name="command",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=["new", "dev", "up", "build", "dockerfile"]
        )
    )
)
print("✓ Added 'command' column")


In [ ]:
# Column 2: Template name for 'new' command (choose from available templates)
config_builder.add_column(
    SamplerColumnConfig(
        name="template",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[
                "basic",
                "react-agent",
                "memory-agent",
                "retrieval-agent",
                "data-enrichment"
            ]
        )
    )
)
print("✓ Added 'template' column")


In [ ]:
# Column 3: Whether user specifies a custom path (25% chance)
config_builder.add_column(
    SamplerColumnConfig(
        name="include_path",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[True, False],
            weights=[1, 3]  # 25% True, 75% False
        )
    )
)
print("✓ Added 'include_path' column")


In [ ]:
# Column 4: Port number (uniform distribution between 3000-9000)
config_builder.add_column(
    SamplerColumnConfig(
        name="port",
        sampler_type=SamplerType.UNIFORM,
        params=UniformSamplerParams(low=3000, high=9000),
        convert_to="int"
    )
)
print("✓ Added 'port' column")


In [ ]:
# Column 5: No-browser flag (20% chance)
config_builder.add_column(
    SamplerColumnConfig(
        name="no_browser",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[True, False],
            weights=[1, 4]  # 20% True, 80% False
        )
    )
)
print("✓ Added 'no_browser' column")


In [ ]:
# Column 6: Watch flag for 'up' command (33% chance)
config_builder.add_column(
    SamplerColumnConfig(
        name="watch",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[True, False],
            weights=[1, 2]  # 33% True, 67% False
        )
    )
)
print("✓ Added 'watch' column")


In [ ]:
# Column 7: Docker image tag
config_builder.add_column(
    SamplerColumnConfig(
        name="image_tag",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[
                "myapp:latest",
                "latest",
                "langgraph-app:v1",
                ""  # Empty = no custom tag
            ]
        )
    )
)
print("✓ Added 'image_tag' column")


In [ ]:
# Column 8: Dockerfile path
config_builder.add_column(
    SamplerColumnConfig(
        name="dockerfile_path",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[
                "Dockerfile",
                "Dockerfile.custom",
                ""  # Empty = default path
            ]
        )
    )
)
print("✓ Added 'dockerfile_path' column")


### LLM-Generated Columns

Now we add the columns that use Nemotron to generate:
1. **Natural language input**: A user request in plain English
2. **Structured JSON output**: The corresponding CLI tool-call


In [ ]:
# Column 9: Natural language query (generated by LLM using the sampled fields)
input_prompt = """You are an IT assistant. A user wants to use LangGraph CLI. 
Their goal: for command '{{ command }}'. 
{% if command == 'new' %}They want to create a new LangGraph project using the '{{ template }}' template\
{{ ' in directory \\'my_project\\'' if include_path else '' }}.\
{% elif command == 'dev' %}\
They want to run the dev server{% if port < 8000 %} on port {{ port }}{% endif %}\
{% if no_browser %} without opening a browser{% endif %}.\
{% elif command == 'up' %}\
They want to launch the server container{% if port >= 8000 %} on port {{ port }}{% endif %}\
{% if watch %} and have it watch for code changes{% endif %}.\
{% elif command == 'build' %}\
They need to build the Docker image{% if image_tag %} with tag '{{ image_tag }}'{% endif %}.\
{% elif command == 'dockerfile' %}\
They want to generate a Dockerfile{% if dockerfile_path and dockerfile_path != 'Dockerfile' %} at '{{ dockerfile_path }}'{% endif %}.\
{% endif %} 
Craft a concise request."""

config_builder.add_column(
    LLMTextColumnConfig(
        name="input",  # User query text
        model_alias="nemotron-nano-v2",  # Nemotron 9B v2 model
        prompt=input_prompt,
        system_prompt="Respond with a one-sentence request, politely phrased, using any details provided."
    )
)
print("✓ Added 'input' column (natural language query)")


In [ ]:
# Column 10: Structured JSON output (LLM generates the CLI tool-call JSON)
output_prompt = """You are a CLI assistant. Convert the user request to a JSON tool-call object for LangGraph CLI. 
Only output valid JSON. Use keys: 'command' (string) and include any relevant options as additional keys.
User wants to: {{ input }}
JSON:"""

config_builder.add_column(
    LLMTextColumnConfig(
        name="output",  # Structured JSON representation
        model_alias="nemotron-nano-v2",
        prompt=output_prompt,
        system_prompt="You output JSON only, with correct keys for the CLI command."
    )
)
print("✓ Added 'output' column (structured JSON)")


## Step 4: Generate Synthetic Data

Now we generate the synthetic dataset using the Data Designer preview endpoint.


In [ ]:
# Generate synthetic data (50 examples for preview)
NUM_RECORDS = 50

print(f"Generating {NUM_RECORDS} synthetic examples...")
print("This may take a few minutes depending on your hardware.\n")

preview = client.preview(config_builder, num_records=NUM_RECORDS)
df = preview.dataset  # Pandas DataFrame of results

print(f"✓ Generated {len(df)} examples")


In [ ]:
# Display sample input/output pairs
print("Sample Input/Output Pairs:\n")
print("=" * 80)

for idx, row in df[['input', 'output']].head(5).iterrows():
    print(f"\n📝 Input:  {row['input']}")
    print(f"💻 Output: {row['output']}")
    print("-" * 80)


In [ ]:
# Show distribution of commands in the dataset
print("\nCommand Distribution:")
print(df['command'].value_counts())


In [ ]:
# Preview the full dataframe
df.head(10)


## Step 5: Export Dataset to JSONL

Export the synthetic dataset in JSONL format for model fine-tuning.


In [ ]:
# Export full dataset to JSONL
output_file = "langgraph_cli_synthetic.jsonl"
df.to_json(output_file, orient="records", lines=True)
print(f"✓ Saved {len(df)} examples to {output_file}")


In [ ]:
# Export only input/output pairs (for training)
training_file = "langgraph_cli_training.jsonl"
df[['input', 'output']].to_json(training_file, orient="records", lines=True)
print(f"✓ Saved training pairs to {training_file}")


In [ ]:
# Preview the exported file
print("\nPreview of exported JSONL:\n")
!head -3 langgraph_cli_training.jsonl


## Step 6: Generate Larger Dataset (Optional)

For production training, you'll want more examples. Adjust `NUM_RECORDS` as needed.


In [ ]:
# Uncomment to generate a larger dataset
# NUM_RECORDS_LARGE = 500
# 
# print(f"Generating {NUM_RECORDS_LARGE} synthetic examples...")
# preview_large = client.preview(config_builder, num_records=NUM_RECORDS_LARGE)
# df_large = preview_large.dataset
# 
# df_large.to_json("langgraph_cli_synthetic_large.jsonl", orient="records", lines=True)
# print(f"✓ Saved {len(df_large)} examples to langgraph_cli_synthetic_large.jsonl")


## Cleanup

When you're done, stop the Data Designer services. See [README.md](./README.md) for cleanup instructions.


In [ ]:
# Stop the Data Designer services (uncomment to run)
# !cd nemo-data-designer-docker-compose_v25.08 && docker compose -f docker-compose.ea.yaml down


---

## Next Steps

1. **Validate the data**: Review the generated examples for quality
2. **Fine-tune a model**: Use the JSONL file to fine-tune a model (e.g., with NeMo Framework)
3. **Integrate with your agent**: Use the trained model to power a LangGraph CLI assistant

### Example Training Commands

```bash
# Using NeMo Framework for fine-tuning
python -m nemo.collections.nlp.models.language_modeling.megatron_gpt_sft \
    --config-path=conf \
    --config-name=megatron_gpt_sft.yaml \
    model.data.train_ds.file_path=langgraph_cli_training.jsonl
```
